## Actividad 2: Scrapear Frases

En esta actividad 2 del módulo 3 scrapearemos frases célebres de la página web http://quotes.toscrape.com

Es importante que nos guiemos de la documentación: https://docs.scrapy.org/en/latest/ para saber qué está ocurriendo en el código y aprender nuevas formas de ejecutar el scraping

Con el código que vemos debajo podemos asegurarnos de instalar scrapy si es que no lo tenemos. Luego procedemos a importar el crawler que utilizaremos.

In [0]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

import logging
import json

Diseñamos las funciones de nuestro Pipeline.
* ```open_spider``` se ejecuta cuando se inicializa el spider.
* ```close_spider``` se ejecuta cuando se cierra el spider.
* ```process_item``` es la función que se llama para procesar el objeto.



In [0]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

Una vez tenemos definida nuestra Pipeline, procedemos a definir nuestro propio Spider. Atención a las ```custom_settings``` y a cómo se nos permite empezar a scrapear en más de una URL con ```start_urls```.

In [0]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING, # Con esto reducimos el número de mensajes que recibimos
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Ejecuta nuestra Pipeline
        'FEED_FORMAT':'json',                                 # Herramienta que nos permite hacer lo mismo que nuestra Pipeline
        'FEED_URI': 'quoteresult.json'
    }
    
    def parse(self, response): # Definición de nuestro proceso de parsing
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

Finalmente ejecutamos el proceso de crawling con nuestro Spider.

In [4]:
process = CrawlerProcess()

process.crawl(QuotesSpider)
process.start()

2019-11-19 02:11:05 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2019-11-19 02:11:05 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.8 (default, Oct  7 2019, 12:59:55) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-11-19 02:11:05 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'quoteresult.json', 'LOG_LEVEL': 30}
